# Tutorial on using Propnet

The following is a tutorial designed to give a base overview of the classes and constructs used in the Propnet project. For each class an example of its construction and base usage is provided.

# Defining a Property Network: Propnet

A Propnet object tells us all about the property types and models currently available for use.

The mappings contained in this object define an interconnected network of materials properties. In this form, the Propnet object can be used to enumerate and analyze links between differnet materials properties.

In [1]:
from propnet.core.graph import Propnet

Propnet is not intended for public use at this time. Functionality might change.



In [2]:
p = Propnet()

You can print Propnet to see the property types and models it supports.

In [3]:
print(p)

Propnet Graph

Quantity Types:
	 Crystallographic structure (oxidation-state decorated)
		 Decorate crystal structure with oxidation state
	 Is Metallic
	 Crystallographic structure
		 Atomic Density Model
	 Formula
	 Crystal Prototype
	 Atomic Density
		 Debye model
	 Elastic tensor (in Voigt notation)
		 Voight's Shear Modulus
		 Hill's Shear Modulus
		 Voight's Bulk Modulus
		 Hill's Bulk Modulus
		 Voight's Shear Modulus
	 Interplanar Spacing
		 Peierls-Nabarro Stress for Dislocation Slip
	 Elasticity-Derived Longitudinal Sound Velocity
		 Calculate Elasticity-Based Sound Velocity (Mean)
	 Final Energy
	 Goldschmidt tolerance factor
	 Final Energy Per Atom
	 Thermal conductivity
		 Clarke thermal conductivity
	 Refractive index
		 Optical Reflectance from the complex refractive index
		 Refractive index, relative permeability and permittivity
	 Ionic radius of B site in perovskite
		 Perovskite Classifier
	 Absorption Coefficient
		 Optical Absorbance from the absorption coefficien

Or you can iterate over the graph to see how it works behind the scenes.

In [4]:
for n in p.graph.nodes():
    print(n)

Symbol<object<structure_oxi>>
Symbol<object<is_metallic>>
Symbol<object<structure>>
Symbol<object<pretty_formula>>
Symbol<object<prototype>>
Symbol<property<atomic_density>>
Symbol<property<elastic_tensor_voigt>>
Symbol<property<interplanar_spacing>>
Symbol<property<sound_velocity_longitudinal>>
Symbol<property<final_energy>>
Symbol<property<goldschmidt_tolerance_factor>>
Symbol<property<final_energy_per_atom>>
Symbol<property<thermal_conductivity>>
Symbol<property<refractive_index>>
Symbol<property<ionic_radius_b>>
Symbol<property<absorption_coefficient>>
Symbol<property<p_wave_modulus>>
Symbol<property<extinction_coefficient>>
Symbol<property<electronic_thermal_conductivity>>
Symbol<property<transmittance>>
Symbol<property<interatomic_spacing>>
Symbol<property<band_gap_pbe>>
Symbol<property<electrical_resistivity>>
Symbol<property<relative_permittivity>>
Symbol<property<snyder_acoustic_sound_velocity>>
Symbol<property<sound_velocity_mean>>
Symbol<property<volume_unit_cell>>
Symbol<pr

# Defining a Symbol or Property

A SymbolType object is used to represent types of properties (such as Young's Modulus) or conditions (such as Temperature).
- All SymbolType objects are accessible in a global DEFAULT_SYMBOL_TYPES variable.
- Various metadata for each SymbolType can be accessed as shown below.

In [5]:
from propnet.symbols import DEFAULT_SYMBOL_TYPES

In [6]:
symbol_type_object = DEFAULT_SYMBOL_TYPES['youngs_modulus']
print(symbol_type_object)

youngs_modulus:
	name:	youngs_modulus
	category:	property
	units:	1.0 gigapascal
	object_type:	None
	display_names:	["Young's modulus", 'Elastic modulus']
	display_symbols:	['E']
	shape:	1
	comment:	



A Symbol object is used to represent values of properties (such as Young's Modulus = 200GPa) or conditions (such as temperature = 300K).

- All Symbol objects have a SymbolType giving the type of property represented by the value.
- All Symbol objects must be created at runtime by specifying a value during instantiation.
- All Symbol objects have a list of strings called "tags" used to further label the property.

In [7]:
from propnet.core.quantity import Quantity

In [8]:
steel_youngs_modulus = Quantity('youngs_modulus', 200, ['mild steel'])
print(steel_youngs_modulus)

<youngs_modulus, 200 gigapascal, ['mild steel']>


# Defining a Material

A Material object is used to represent a collection of information known about a given material.

When it is first created it has no information; however, properties can be added to the material one-by-one.

In [9]:
from propnet.core.materials import Material
from propnet.core.symbols import Symbol

In [10]:
mild_steel = Material()
youngs_modulus = Quantity('youngs_modulus', 200, [])
mild_steel.add_property(youngs_modulus)
print(mild_steel)

Material: c72d0aab-b4a6-4aea-822c-bdceb9c8f6e5
	youngs_modulus:	200 gigapascal



# Combining Models, Materials, and Symbols

As illustrated, a Propnet object contains information for connecting many different models and symbol types. This forms an abstract web of interconnected variables without any quantities specified.

On the other hand, a Material object represents a grouping of values for different variables. These are represented as a collection of Symbol objects identified with the material.

At runtime, a single Propnet object can be combined with one or more Material objects. This procedure allows values to be plugged in to variables. Assuming the required inputs for a model all have values, the Propnet object can then dynamically predict the values for the output variables of the model.

In [11]:
p = Propnet()

silica = Material()
refractive_index = Quantity('refractive_index', 1.458, [])
relative_permittivity = Quantity('relative_permittivity', 3.9, [])

silica.add_property(refractive_index)
silica.add_property(relative_permittivity)

p.add_material(silica)
print(silica)

Material: c80839f5-9f1c-4b75-8d95-65e17f64278e
	refractive_index:	1.458 dimensionless
	relative_permittivity:	3.9 dimensionless



Propnet can now examine the input values and identify if any models can be used to derive additional properties.

In this example, we've provided the relative permittivity and index of refraction of silica. Thus, using the canonical relationship from electromagnetism, we expect Propnet to properly derive the relative permeability.

Re-examining the material object previously created, a new Symbol object, the relative permeability, is now associated with that material.

In [12]:
p.evaluate(material=silica)
print(silica)

Material: c80839f5-9f1c-4b75-8d95-65e17f64278e
	refractive_index:	1.458 dimensionless
	relative_permittivity:	3.9 dimensionless
	relative_permeability:	0.545067692307692 dimensionless



# Working with Models

A Model object is used to represent a relationship between different materials property variables. This object can be directly manipulated and stores relavent metadata available as direct attributes.

- All Models are imported as classes at runtime.
- A Model class must be instantiated to be used at runtime.

In [13]:
from propnet.models import *
model = RefractiveIndexfromRelPerm()
print(model.description)
print(model.name)
print()
print(model.equations)


The refractive index gives the factor by which the speed of light is reduced in a medium.

Likewise, modeling the induced magnetic and electric dipoles as linear within a material,
a relative spatial electrical permittivity and relative spatial magnetic permeability
arise from consideration of the total electrical and magnetic fields.

From the Maxwell Relations, the index of refraction is equal to the geometric mean  of the
relative permittivity and the relative permeability.

RefractiveIndexfromRelPerm

['n - sqrt(Ur*Er)']


The Model class is a generally-defined interface, and subclasses may alter many aspects of its underlying functionality.


Most Model objects will contain equations, symbols, and connections attributes. These define the core functionality of the model:

The equations attribute will contain a list of sympy-parsable expressions. These expressions imply trivial equations such that the expression is equal to zero.

The symbols attribute map the symbols used in the equations to Symbol_Type objects used in the Property Network.

The connections attribute shows what outputs can be generated from a set of inputs.

In [14]:
print(model.symbol_mapping)
print()
print(model.equations)
print()
print(model.connections)

{'Er': 'relative_permeability', 'Ur': 'relative_permittivity', 'n': 'refractive_index'}

['n - sqrt(Ur*Er)']

[{'inputs': ['Ur', 'Er'], 'outputs': ['n']}, {'inputs': ['Er', 'n'], 'outputs': ['Ur']}, {'inputs': ['Ur', 'n'], 'outputs': ['Er']}]


A Model can be evaluated to generate outputs if given a complete set of inputs.

Given the relative permeability and permittivity, the Refractive Index From Relative Permeability model can correctly calculate the index of refraction.

This is given by 'n' in the dictionary below.

In [15]:
model.evaluate({'Ur': 0.54, 'Er': 3.9})

{'n': <Quantity(1.45120639469374, 'dimensionless')>, 'successful': True}

# Loading Materials Data

Material properties can be loaded in from the Materials Project so they don't need to be defined and added manually.

Accessing Materials Project data requires an API key. You must input your own API key below to run the sample. You can locate your api key by logging into materialsproject.org and visiting the dashboard.

In [16]:
from propnet.ext.matproj import *
my_api_key = None
silica = import_material('mp-546794', api_key=my_api_key)
print(silica)

Material: 55a6cdb6-dd27-4fa3-afc3-b44445e47b67
	structure:	Full Formula (Si2 O4)
Reduced Formula: SiO2
abc   :   5.138209   5.139163   5.138919
angles:  88.540850 120.840203 120.841618
Sites (6)
  #  SP           a         b         c    coordination_no  forces
---  ----  --------  --------  --------  -----------------  --------------------------------------
  0  O     0.334692  0.375014  0.209679                  4  [0.00907491, -0.00223112, 0.00638321]
  1  Si    0.999993  1.00001   0.999974                  4  [0.00399411, 0.00268543, 0.00019724]
  2  O     0.834699  0.959681  0.625017                  4  [0.00353084, 0.00703423, -0.00908806]
  3  O     0.165321  0.790307  0.12502                   4  [-0.00778536, -0.00920807, 0.00757932]
  4  O     0.665314  0.875015  0.040304                  4  [-0.01303024, -0.000511, -0.00656695]
  5  Si    0.49998   0.749969  0.250005                  4  [0.00421573, 0.00223052, 0.00149524]
	lattice_unit_cell:	[[ 4.59584983 -0.71705099 -2.182

# Working with Units

## Symbol Objects

No units need to be specified when defining a Symbol or plugging values into Models. In the case that no units are supplied, default units are inferred based on those provided in the corresponding SymbolType object.

For instance, youngs_modulus is a SymbolType with the default units of gigapascal. 
In the example above:

           youngs_modulus = Symbol('youngs_modulus', 200, [])
           
youngs_modulus automatically has the value 200 gigapascals despite no units being provided.

It is possible to provide your own units by working with the <b>unit registry</b> (ureg). The examples below demonstrate this capability using the method: 
            
            ureg.Quantity(value, unit)
            
The property network will automatically convert the units you supply into canonical units behind the scenes for use in models that use the property.

In [17]:
from propnet import ureg
p = Propnet()

mild_steel = Material()
yield_strength = Quantity('yield_stress', ureg.Quantity(53700, 'psi'), [])
mild_steel.add_property(yield_strength)
print(mild_steel)

Material: 18c0a49a-ee61-4937-a948-dcc75acd96b9
	yield_stress:	0.3702484666431411 gigapascal



## Models
No units need to be specified when plugging values into a Model. In the case that no units are supplied, default units are inferred based on those provided in the corresponding SymbolType objects.

            model.evaluate(dict)
            
The above method can be used to plug in raw python floats, indicating that default units are implied. 

The above method can also be used with custom units supplied to one or more of the values in the dictionary. 

In the example below we use the property network to calculate the Peierls-Nabarro Stress for a disloaction on the (111) plane in Aluminum (FCC) using custom units of psi for shear modulus.

In [18]:
model = PeierlsStress()
G = ureg.Quantity(3.9*10**6, 'psi')
a = 4.046 / 3**(1/2)  # Angstroms automatically assumed by the model.
b = 4.046 / 2**(1/2)  # Angstroms automatically assumed by the model.
nu = 0.33

tau = model.evaluate({'G': G, 'a': a, 'b': b, 'nu': nu})
print(tau['T_pn'])

0.0127112280131544 gigapascal


## Converting Units
Given you have an output with units, it is straightforward to convert between units using Pint.

In [19]:
# Converting to megapascals.
print(tau['T_pn'].to('megapascal'))

12.7112280131544 megapascal


# Creating Custom Models and Properties

The property network comes with many different models and properties pre-loaded and ready for use.

To add additional properties to the project can be accomplished in two different forms:

- Defining additional .yaml and .py files in the propnet.models and propnet.symbols folders respectively. For these changes to take effect, the property network must be re-loaded.

- Defining additional Model and SymbolType classes at runtime.

Both approaches will be detailed below.

## Defining new model files:
Models require a model_name.yaml and a model_name.py file to be defined in the propnet/models folder.

The syntax for a model.yaml definition is as follows:

<table style="width:100%">
  <tr>
    <th>Field Name</th>
    <th>Field Format</th> 
    <th>Field Description</th>
  </tr>
  <tr>
    <td>title</td>
    <td>string</td> 
    <td>title of the model</td>
  </tr>
  <tr>
    <td>tags</td>
    <td>list of strings</td> 
    <td>any user-specific tags applying to the model</td>
  </tr>
  <tr>
    <td>references</td>
    <td>list of strings</td> 
    <td>any urls for citing the model</td>
  </tr>
  <tr>
    <td>symbol_mapping</td>
    <td>dictionary, string keys, string values</td> 
    <td>mapping from symbols used in the model to SymbolType names used in the property network</td>
  </tr>
  <tr>
    <td>connections</td>
    <td>list of dictionaries with keys: 'inputs', 'outputs' mapping to lists of symbol strings used in the model</td> 
    <td>datastrucutre used to determine the set of all outputs that can be generated from a set of inputs</td>
  </tr>
  <tr>
    <td>equations</td>
    <td>list of strings</td> 
    <td>list of equations that create the model, defined as the value of the expression equals zero</td>
  </tr>
</table>

By default the python file can be a simple class declaration:

            class model_name(AbstractModel):
                pass
                
Methods of the AbstractModel class can be overridden to allow for custom model evaluation() or constraint handling (see below).

## Defining new SymbolType files:
SymbolTypes require a symbol_type.yaml file to be defined in the propnet/symbols folder.
The syntax for a symbol_type.yaml definition is as follows:
<table style="width:100%">
  <tr>
    <th>Field Name</th>
    <th>Field Format</th> 
    <th>Field Description</th>
  </tr>
  <tr>
    <td>name</td>
    <td>string</td> 
    <td>name of the property, must match the file name</td>
  </tr>
  <tr>
    <td>units</td>
    <td>nested list: [1.0, [[unit name string, unit_power], ...]</td> 
    <td>the units used to represent this SymbolType, adjacent entries in the list indicate multiplication.</td>
  </tr>
  <tr>
    <td>display_names</td>
    <td>list of strings</td> 
    <td>any strings useful for representing this SymbolType</td>
  </tr>
  <tr>
    <td>display_symbols</td>
    <td>list of strings</td> 
    <td>any short string symbols useful for representing this SymbolType</td>
  </tr>
  <tr>
    <td>dimension</td>
    <td>list of numbers</td> 
    <td>datastructure to give the length of each dimension in a higher order tensor representation of the property, 1 for a scalar value.</td>
  </tr>
  <tr>
    <td>comment</td>
    <td>string</td> 
    <td>any comments pertinent to the SymbolType</td>
  </tr>
  <tr>
    <td>category</td>
    <td>string</td> 
    <td>whether this is a property, object, or condition</td>
  </tr>
</table>

## Defining Models at Runtime:
A Model can be defined as a new class that extends the AbstractModel class.

See example below for a simple overview. All information is loaded in through the metadata optional argumnent. This defines a model that takes in values (a, b) and returns their product.

Based on symbol_mapping, this model expects variable a to be of type A, variable b to be of type B, and outputs a variable c of type C.

For more advanced behavior, additional methods of AbstractModel could be overridden for custom model evaluation() or constraint handling (see below).

In [20]:
from propnet.core.models import AbstractModel
class Model1 (AbstractModel):
    def __init__(self, symbol_types=None):
        AbstractModel.__init__(self, metadata={
                        'title': 'model1',
                        'tags': [],
                        'references': [],
                        'symbol_mapping': {'a': 'A',
                                           'b': 'B',
                                           'c': 'C'
                                           },
                        'connections': [{
                                         'inputs': ['a', 'b'],
                                         'outputs': ['c']
                                         }],
                        'equations': ['c=a*b'],
                        'description': ''
            },
            symbol_types=symbol_types)

## Defining SymbolTypes at Runtime:
A new SymbolType can be defined as a new instance of the SymbolType class by specifying all the metadata present in a symbol_type.yaml file.

See example below.

In [21]:
from propnet.core.symbols import Symbol
A = Symbol('gibbs_free_energy', ['G'], ['G'], units=[1.0, [['joules',1]]], shape=1)

# A Grand Example

In [22]:
silicon = Material()
silicon.add_property(Quantity('temperature', 300, []))
silicon.add_property(Quantity('band_gap', 1.12, []))
silicon.add_property(Quantity('electrical_conductivity', 4.34E-4, []))

aluminum = Material()
aluminum.add_property(Quantity('temperature', 300, []))
aluminum.add_property(Quantity('band_gap', 0, []))
aluminum.add_property(Quantity('electrical_conductivity', 3.69E7, []))

p1 = Propnet(materials=[silicon])
p1.evaluate()

p2 = Propnet(materials=[aluminum])
p2.evaluate()

print("Silicon's Graph")
print(silicon)
print()
print("Aluminum's Graph")
print(aluminum)

Silicon's Graph
Material: b867754d-8b8e-4e9f-a3b3-2a5894341fe4
	temperature:	300 kelvin
	band_gap:	1.12 electron_volt
	electrical_conductivity:	0.000434 / meter / ohm
	band_gap_pbe:	1.50816023738872 electron_volt
	band_gap_gw:	1.13627254509018 electron_volt
	is_metallic:	0 dimensionless


Aluminum's Graph
Material: d367675e-899b-4597-a1f7-80cf404ecc4f
	temperature:	300 kelvin
	band_gap:	0 electron_volt
	electrical_conductivity:	36900000.0 / meter / ohm
	band_gap_pbe:	0.677299703264095 electron_volt
	band_gap_gw:	0.0140280561122244 electron_volt
	is_metallic:	1 dimensionless
	electronic_thermal_conductivity:	271.215000000000 watt / kelvin / meter
	electrical_resistivity:	2.71002710027100E-8 meter * ohm



# Models with Custom Evaluation

In several cases, a model may not be representable as an equation parsable by sympy. In these cases, the plug_in method must be overridden to perform the custom evaluation. 

The is_metallic model is reproduced below, yielding a simple example of custom evaluation.

In [23]:
class IsMetallic(AbstractModel):
    def plug_in(self, symbol_values):
        return {'is_metallic': symbol_values['E_g'] <= 0}

# A Grand Example